In [1]:
import os

In [2]:
%pwd

'e:\\Projects\\Text-Summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Projects\\Text-Summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [9]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

c:\Users\admin\anaconda3\envs\summary\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        try:
            input_encodings = self.tokenizer(
                example_batch['dialogue'],
                max_length=1024,
                truncation=True,
                padding="max_length"
            )

            with self.tokenizer.as_target_tokenizer():
                target_encodings = self.tokenizer(
                    example_batch['summary'],
                    max_length=128,
                    truncation=True,
                    padding="max_length"
                )

            return {
                'input_ids': input_encodings['input_ids'],
                'attention_mask': input_encodings['attention_mask'],
                'labels': target_encodings['input_ids']
            }

        except Exception as e:
            logger.error(f"Error while tokenizing batch: {e}")
            raise e

    def convert(self):
        try:
            logger.info(f"Loading dataset from: {self.config.data_path}")
            dataset_samsum = load_from_disk(self.config.data_path)

            logger.info("Starting tokenization process...")
            dataset_samsum_pt = dataset_samsum.map(
                self.convert_examples_to_features,
                batched=True,
                batch_size=16
            )

            save_path = os.path.join(self.config.root_dir, "samsum_dataset")
            dataset_samsum_pt.save_to_disk(save_path)
            logger.info(f"Tokenized dataset saved successfully at: {save_path}")

        except Exception as e:
            logger.error(f"Error in data transformation: {e}")
            raise e


In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    
    print("Starting data transformation...")
    data_transformation.convert()
    print("Data transformation completed successfully!")

except Exception as e:
    import traceback
    print("Error during data transformation:")
    print(traceback.format_exc())


YAML file loaded successfully: config\config.yaml
[2025-11-01 23:28:33,631: INFO: common: YAML file loaded successfully: config\config.yaml]
YAML file loaded successfully: params.yaml
[2025-11-01 23:28:33,648: INFO: common: YAML file loaded successfully: params.yaml]
Created directory at: artifacts
[2025-11-01 23:28:33,654: INFO: common: Created directory at: artifacts]
Created directory at: artifacts/data_transformation
[2025-11-01 23:28:33,660: INFO: common: Created directory at: artifacts/data_transformation]
Starting data transformation...
Loading dataset from: artifacts/data_ingestion/samsum_dataset
[2025-11-01 23:28:38,405: INFO: 995596329: Loading dataset from: artifacts/data_ingestion/samsum_dataset]
Starting tokenization process...
[2025-11-01 23:28:39,166: INFO: 995596329: Starting tokenization process...]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\admin\anaconda3\envs\summary\lib\site-packages\transformers\tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 10287.46 examples/s]

Tokenized dataset saved successfully at: artifacts/data_transformation\samsum_dataset
[2025-11-01 23:29:06,001: INFO: 995596329: Tokenized dataset saved successfully at: artifacts/data_transformation\samsum_dataset]
Data transformation completed successfully!
